In [1]:
import os
import gc
import json
import pathlib
from datetime import datetime
import optuna
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import roc_auc_score
import torch
from typing import List, Dict, Union, Tuple, NamedTuple
from tqdm import tqdm
import scml
from scml import pandasx as pdx
tim = scml.Timer()
tim.start()
os.environ["TOKENIZERS_PARALLELISM"] = "false"
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()

In [2]:
ts = datetime.now().strftime('%Y%m%d_%H%M%S')
job_dir = f"models/xgb/{ts}"
pathlib.Path(job_dir).mkdir(parents=True, exist_ok=True)
num_boost_round: int = 100
lr: Tuple[float, float] = (1e-3, 1e-3)
feature_fraction: Tuple[float, float] = (1, 1)
min_data_in_leaf: Tuple[int, int] = (20, 20)
objective: str = "binary:logistic"
n_trials: int = 1
label = "generated"

In [3]:
df = pd.read_parquet("input/features.parquet")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221612 entries, 0 to 221611
Columns: 50047 entries, row_id to tf_Ġyoutube_Ġuploaded_Ġby
dtypes: float32(50033), int16(2), int32(5), int8(1), object(6)
memory usage: 41.3+ GB


In [4]:
features = []
prefixes = ["ch_", "ws_", "ts_", "va_", "tf_"]
for col in df.columns:
    for prefix in prefixes:
        if col.startswith(prefix):
            features.append(col)
features.sort()
print(f"{len(features)} features\n{features[:100]}")

50036 features
['ch_digit_frac', 'ch_len', 'ch_letter_frac', 'ch_punc_frac', 'ch_repeat_char_frac', 'ch_space_frac', 'ch_upper_frac', 'tf_1_n_1', 'tf_a_Ġet_Ġal', 'tf_a_Ġis_Ġa', 'tf_ability_Ġof_Ġthe', 'tf_ality_Ġof_Ġthe', 'tf_an_Ġet_Ġal', 'tf_ance_Ġof_Ġthe', 'tf_ar_Ġet_Ġal', 'tf_as_Ġet_Ġal', 'tf_ated_Ġby_Ġthe', 'tf_ated_Ġin_Ġthe', 'tf_ation_Ġof_Ġthe', 'tf_ations_Ġof_Ġthe', 'tf_e_Ġet_Ġal', 'tf_ed_Ġat_Ġthe', 'tf_ed_Ġby_Ġthe', 'tf_ed_Ġfrom_Ġthe', 'tf_ed_Ġin_Ġthe', 'tf_ed_Ġinto_Ġthe', 'tf_ed_Ġon_Ġthe', 'tf_ed_Ġto_Ġthe', 'tf_ed_Ġwith_Ġthe', 'tf_el_Ġet_Ġal', 'tf_en_Ġet_Ġal', 'tf_er_Ġet_Ġal', 'tf_er_Ġin_Ġthe', 'tf_ers_Ġin_Ġthe', 'tf_ers_Ġof_Ġthe', 'tf_es_Ġet_Ġal', 'tf_es_Ġin_Ġthe', 'tf_es_Ġof_Ġthe', 'tf_i_Ġet_Ġal', 'tf_ia_Ġet_Ġal', 'tf_ification_Ġof_Ġthe', 'tf_in_Ġet_Ġal', 'tf_in_Ġin_Ġthe', 'tf_ing_Ġin_Ġthe', 'tf_ing_Ġof_Ġthe', 'tf_ing_Ġto_Ġthe', 'tf_is_Ġet_Ġal', 'tf_isation_Ġof_Ġthe', 'tf_ism_Ġand_Ġthe', 'tf_ism_Ġin_Ġthe', 'tf_ists_Ġin_Ġthe', 'tf_ities_Ġof_Ġthe', 'tf_ity_Ġof_Ġthe', 'tf_ivenes

In [5]:
#X_train, X_test, y_train, y_test = train_test_split(tra[features], tra[label], test_size=0.2)

th = 0.29
tra = df[df["white_sim"]>=th].copy()
val = df[df["white_sim"]<th].copy()
t = len(tra)
v = len(val)
n = t+v
print(f"val%={v/n:.4f}, len(tra)={t:,}, len(val)={v:,}")
del df
gc.collect()
dtrain = xgb.DMatrix(tra[features], tra[label], enable_categorical=False)
dval = xgb.DMatrix(val[features], val[label], enable_categorical=False)
pdx.value_counts(val[label])

val%=0.0473, len(tra)=211,120, len(val)=10,492


,count,percent
generated,,
0,10451,0.996092
1,41,0.003908


In [6]:
%%time
model = xgb.train(
   params={
       "objective": objective,
       "learning_rate": 0.2,
       "min_child_weight": 40,
       "colsample_bytree": 0.75,
       "max_depth": 8,
   },
   dtrain=dtrain,
   num_boost_round=2000,
   evals=[(dtrain, "train"), (dval, "val")],
   verbose_eval=40,
   early_stopping_rounds=100,
)
print(f"best score {model.best_score:.5f} at iteration {model.best_iteration}")
model.save_model(f"{job_dir}/model.json")

[0]	train-logloss:0.25120	val-logloss:0.14830
[40]	train-logloss:0.04680	val-logloss:0.00976
[80]	train-logloss:0.03365	val-logloss:0.00812
[120]	train-logloss:0.02828	val-logloss:0.00759
[160]	train-logloss:0.02463	val-logloss:0.00730
[200]	train-logloss:0.02218	val-logloss:0.00722
[240]	train-logloss:0.01995	val-logloss:0.00724
[280]	train-logloss:0.01817	val-logloss:0.00730
[302]	train-logloss:0.01725	val-logloss:0.00729
best score 0.00718 at iteration 203
CPU times: user 6h 19min 45s, sys: 19min 54s, total: 6h 39min 39s
Wall time: 30min 32s


In [7]:
%%time
y_true = val[label].tolist()
y_pred = model.predict(data=dval, iteration_range=(0, model.best_iteration+1))
auc = roc_auc_score(y_true, y_pred, average="macro")
print(f"auc={auc:.4f}")
print(f"y_pred={y_pred.shape}\n{y_pred[:5]}")

auc=0.9940
y_pred=(10492,)
[0.38793376 0.00083771 0.00339701 0.4036299  0.8062436 ]
CPU times: user 1.28 s, sys: 796 ms, total: 2.08 s
Wall time: 160 ms


In [8]:
%%time
scores = model.get_score(importance_type="gain")
assert len(scores)!=0
rows = []
for feature, score in scores.items():
    rows.append({'importance': score, 'feature': feature})
idf = pd.DataFrame.from_records(rows)
idf = idf.sort_values(["importance"], ascending=False, ignore_index=True)
fp = f"{job_dir}/importance.csv"
idf.to_csv(fp, index=True)
print(f"Saved {fp}")
idf.T.head()

Saved models/xgb/20240123_055726/importance.csv
CPU times: user 47.8 ms, sys: 99.9 ms, total: 148 ms
Wall time: 12.8 ms


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427
importance,524.765625,511.723145,463.868713,433.928864,420.624207,411.376556,376.831909,369.976868,353.973694,303.72998,300.026978,292.972626,273.50116,251.780762,244.906372,236.761963,233.246536,233.063965,221.298813,219.814575,210.735504,203.567871,203.105133,193.787521,192.337662,184.755463,183.179443,179.763107,175.643707,174.943481,173.67244,173.267426,170.513885,168.751465,159.221619,145.114731,130.477539,127.348206,127.241875,126.852661,120.087181,119.284294,118.40284,117.862144,116.257812,113.81414,113.317337,111.852013,110.475929,109.773201,107.882355,105.051758,103.165573,100.473457,99.323547,98.405632,95.234634,94.904297,93.865631,93.012596,91.941025,91.332726,91.285942,90.207336,88.597588,88.547523,88.102951,87.686523,86.764938,85.344482,84.515648,84.465714,82.509758,81.518547,80.563408,79.150604,78.165283,77.331863,76.980949,75.765114,75.130424,73.041496,72.34819,71.464958,71.115196,71.012543,70.588257,69.93084,69.667213,69.306229,69.281212,68.988899,67.249023,65.599655,65.349152,64.613182,64.534729,60.430954,59.935326,59.607166,58.929836,58.499546,58.268459,58.234737,56.726597,54.258781,54.163208,53.991737,52.952942,52.794891,51.508415,51.408188,51.156036,51.139626,50.724384,50.72052,50.342949,50.043159,50.038261,49.429996,49.394344,49.223888,48.869247,48.663818,48.474545,47.103096,46.62048,45.669006,45.539234,45.451225,44.604065,44.273502,43.827038,43.379868,43.36282,42.854084,42.773014,42.372299,42.226044,41.990639,41.539547,41.212307,41.129192,40.773094,40.742634,40.605923,39.848595,39.820763,38.893349,37.985046,37.832687,37.756157,37.26749,36.859478,36.625847,35.936344,35.843155,34.803265,34.671238,34.4146,34.061047,32.980579,32.794315,32.75325,32.717041,31.897552,31.12476,30.753656,30.680832,30.667295,30.456955,30.144304,30.054623,30.03529,29.197378,28.645521,28.385344,27.851171,27.672712,27.265966,26.963526,26.73008,25.991659,25.90309,25.696648,25.401455,25.369764,25.284649,25.201649,25.158321,24.689684,23.954996,23.879873,23.708439,23.152313,22.931667,22.556648,22.513037,22.4916,22.263689,22.114256,22.103745,21.665476,21.634214,21.068371,20.703932,20.572317,20.049183,20.02701,19.925823,19.780899,19.773476,19.742691,19.655207,19.496063,19.416416,19.409914,19.35713,19.056726,18.957182,18.927912,18.691212,18.581039,18.419609,18.121786,17.723103,17.592213,17.568398,17.342649,17.34165,17.150335,17.149137,17.036081,16.875542,16.707352,16.54

In [9]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

Total time taken 0:33:27.300883
